In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt

# Header
- Jump to [Train data](#Train-data)
- Jump to [case_num summarization](#case_num-summarization)
- Jump to [How many feature_texts are shared across case_num?](#How-many-feature_texts-are-shared-across-case_num?)
- Jump to [Test data](#Test-data)
- Jump to [patients_note.csv](#patients_note.csv)

# Train data
Return to [Header](#Header)

In [ ]:
train = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/train.csv")
ptnote = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv")
feat = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/features.csv")

In [ ]:
def mapper(xx: List[str]) -> List[List[Tuple]]:
    res = []
    for x in xx:
        #res.append(list(map(int, x.split(" "))))
        xs = x.split(";")
        inner_res = []
        for x in xs:
            inner_res.append(tuple(map(int, x.split(" "))))
        res.append(inner_res)
    return res

def rubric_retriever(location: List[List[Tuple]], pn_history: str) -> List[str]:
    res = []
    for rubric in location:
        span_txt = ""
        for span_pos in rubric:
            span_txt += pn_history[span_pos[0]:span_pos[1]] + " "
        span_txt = span_txt.strip(" ")
        res.append(span_txt)

    return res



def merge_df(df: pd.DataFrame, ptnote: pd.DataFrame, feat: pd.DataFrame, train=False) -> pd.DataFrame:
    shape_before = df.shape
    df = pd.merge(df, ptnote, on=["pn_num", "case_num"], how="left")
    shape_after = df.shape
    assert (shape_before[0]==shape_after[0])&(shape_before[1]<shape_after[1])

    shape_before = df.shape
    df = pd.merge(df, feat, on=["case_num", "feature_num"], how="left")
    shape_after = df.shape
    assert (shape_before[0]==shape_after[0])&(shape_before[1]<shape_after[1])
    
    if train:
        df["location"] = df["location"].apply(lambda x: mapper(eval(x)))
        df["annotation_manual"] = df.apply(lambda x: rubric_retriever(x["location"], x["pn_history"]), axis=1)
        df["annotation"] = df["annotation"].apply(lambda x: eval(x))
        df["correct_annotation"] = df["annotation"] == df["annotation_manual"]
    
    return df


In [ ]:
print(train.shape)
train = merge_df(train, ptnote, feat, train=True)
print(train.shape)

In [ ]:
display(train.info())
display(ptnote.info())
display(feat.info())

In [ ]:
# print(train.shape)
# train = pd.merge(train, ptnote, on=["pn_num", "case_num"], how="left")
# print(train.shape)

In [ ]:
# print(train.shape)
# train = pd.merge(train, feat, on=["case_num", "feature_num"], how="left")
# print(train.shape)

In [ ]:
# train

In [ ]:
# Length of pn_history
train["pn_history"].apply(lambda x: len(x)).plot.hist();

In [ ]:
i = 0
print(train["pn_history"][i][696:724], ";", train["annotation"][i])

In [ ]:
i = 14299
print(train["pn_history"][i][314:330], ";", train["annotation"][i])

In [ ]:
# tuple(map(int, '136 192'.split(";")[0].split(" ")))

In [ ]:
# '231 240;246 251;260 269;275 280'.split(";")

In [ ]:
# i = 4142
# xx = eval(train["location"][i])
# print(xx)

# def mapper(xx: List[str]) -> List[List[Tuple]]:
#     res = []
#     for x in xx:
#         #res.append(list(map(int, x.split(" "))))
#         xs = x.split(";")
#         inner_res = []
#         for x in xs:
#             inner_res.append(tuple(map(int, x.split(" "))))
#         res.append(inner_res)
#     return res

In [ ]:
# output = mapper(xx)
# output

In [ ]:
i = 4142
print(train.loc[i, "annotation"])
print()
print(train.loc[i, "location"])
print()
print(train.loc[i, "pn_history"])
print()
print(train.loc[i, "pn_history"][136:192])
print(train.loc[i, "pn_history"][231:240])

In [ ]:
# train["location"] = train["location"].apply(lambda x: mapper(eval(x)))
# train["location"]

In [ ]:
train.loc[i, "location"]

In [ ]:
# i = 4142
# res = []
# for rubric in train.loc[i, "location"]:
#     span_txt = ""
#     for span_pos in rubric:
#         span_txt += train.loc[i, "pn_history"][span_pos[0]:span_pos[1]] + " "
#     span_txt = span_txt.strip(" ")
#     res.append(span_txt)
    
# assert res == eval(train.loc[i, "annotation"])

In [ ]:
# train["pn_history"][0]

In [ ]:
# def rubric_retriever(location: List[List[Tuple]], pn_history: str) -> List[str]:
#     res = []
#     for rubric in location:
#         span_txt = ""
#         for span_pos in rubric:
#             span_txt += pn_history[span_pos[0]:span_pos[1]] + " "
#         span_txt = span_txt.strip(" ")
#         res.append(span_txt)

#     return res

In [ ]:
# train["annotation_manual"] = train.apply(lambda x: rubric_retriever(x["location"], x["pn_history"]), axis=1)
# # reference: How to apply a function to two columns of Pandas dataframe https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe
# train[["annotation", "annotation_manual"]]

In [ ]:
#eval(train["annotation"][0])

In [ ]:
train["annotation_manual"][0]

In [ ]:
# train["annotation"] = train["annotation"].apply(lambda x: eval(x))

In [ ]:
# train[["annotation", "annotation_manual"]]

In [ ]:
# train["correct_annotation"] = train["annotation"] == train["annotation_manual"]
train["correct_annotation"].mean()

In [ ]:
train.loc[train["correct_annotation"]!=1, ["annotation", "annotation_manual", "location"]]

In [ ]:
i = 3124
train.loc[i, "pn_history"][549:558]
# should be 549:557

In [ ]:
i = 13297
train.loc[i, "pn_history"][182:221]

In [ ]:
i = 13297
train.loc[i, "pn_history"][182:213] +" "+ train.loc[i, "pn_history"][225:236]
# should be 225:234

In [ ]:
i = 13299
train.loc[i, "pn_history"][409:420]
# should be 409:418

In [ ]:
# words like \r or \n exist everywhere in pn_history, so removing them form pn_history 
# will occur more location mismatches
train["pn_history"][0]

In [ ]:
# Maybe, correcting locations of these three instances is cheaper and valid
# for i = 3124, location 549:558 should be 549:557
# for i = 13297, location 225:236 should be 225:234
# for i = 13299, location 409:420 should be 409:418

# ToDo: above

# [Update]
# Other people found those as well as more incorrect instances using tokenizer
# https://www.kaggle.com/yasufuminakama/nbme-deberta-base-baseline-train/comments#1681447

In [ ]:
print(train["feature_text"].nunique())
print(train["feature_num"].nunique())

In [ ]:
s = train.groupby("feature_num")["feature_text"].nunique()
s[s!=1]

In [ ]:
train["feature_text"].value_counts().plot.hist();

In [ ]:
train["feature_num"].value_counts().plot.hist();

In [ ]:
train.loc[train["feature_num"]==0, "feature_text"].nunique()

In [ ]:
s = train.groupby("feature_text")["feature_num"].nunique()
s = s[s > 1]
s

In [ ]:
# Multiple feature_nums of one feature_text
for idx in s.index:
    print(idx)
    print(train.loc[train["feature_text"]==idx, "feature_num"].unique())
    print()

In [ ]:
for idx in s.index:
    plt.figure()
    plt.title(idx)
    plt.xlabel("feature_num")
    train.loc[train["feature_text"]==idx, "feature_num"].plot.hist();

In [ ]:
train["case_num"].plot.hist();

In [ ]:
res = []
for v in train.loc[train["feature_text"]=="Female", "annotation"]:
    res.extend(v)
res = np.unique(res)
print(res)

In [ ]:
train.loc[train["feature_text"]=="Female", "pn_history"].nunique()

In [ ]:
# Check spelliing variants of annotations for each feature_text
ex_res = {}
for i,ft in enumerate(train["feature_text"].unique()):
    print(f"#{i} {ft}")
    num_instances = train.loc[train["feature_text"]==ft, "pn_history"].nunique()
    print("#instances: ", num_instances)
    
    plt.figure()
    plt.title(ft)
    plt.xlabel("Length of pn_history")
    train.loc[train["feature_text"]==ft, "pn_history"].apply(lambda x: len(x)).plot.hist(bins=100);
    
    res = []
    num_deficiency = 0
    for v in train.loc[train["feature_text"]==ft, "annotation"]:
        if len(v)==0:
            num_deficiency += 1
        res.extend(v)
    res = np.unique(res)
    
    defic_rate = num_deficiency / num_instances
    print(f"%deficiency: {int(defic_rate * 100)}%")
    print(res)
    print()
    #ex_res.append(defic_rate)
    ex_res[defic_rate] = {ft:res}

In [ ]:
len(ex_res)

In [ ]:
# Frequency of deficiency rate
plt.hist(ex_res, bins=50);

In [ ]:
idx = np.argsort(list(ex_res.keys()))
idx

In [ ]:
np.array(list(ex_res.items()))[idx]

In [ ]:
print(len(train))
train["pn_history"].nunique()

# It means that same pn_history is used for multiple feature_texts

In [ ]:
train[train["pn_history"].duplicated()]

In [ ]:
ft = "Meningococcal-vaccine-status-unknown"
train.loc[train["feature_text"]==ft, "annotation"].to_list()

In [ ]:
train.to_csv("train.csv", index=False)

## case_num summarization
Return to [Header](#Header)

In [ ]:
# for i in range(train["case_num"].nunique()):
#     sub = train.loc[train["case_num"]==i, ["pn_history"]].drop_duplicates().reset_index(drop=True)
#     for ph in sub["pn_history"]:
#         print("case_num: ", i)
#         print("pn_history:\n", ph)
#         print()

In [ ]:
sub = train.drop_duplicates(subset=["case_num","feature_text"]).reset_index(drop=True)
s = sub.groupby(["case_num"]).apply(lambda r: r["feature_text"].values)
for i,fts in zip(s.index, s):
    print("case_num: ", i)
    print(" list of feature_text:\n", fts)
    print()

## How many feature_texts are shared across case_num?
Return to [Header](#Header)

In [ ]:
s = train.groupby(["feature_text"])["case_num"].nunique()
s.plot.hist();

In [ ]:
# Number of feature_text
len(s.index)

In [ ]:
# List of feature_text
for ft in s.index:#.sort_values():
    print(ft)

In [ ]:
# Shared feature_text (exact string match)
# Is it possible to use some standardized syndrome code (e.g., ICD) to get fuzzy match?
shared_fts = s[s>1].index.tolist()
print(shared_fts)

In [ ]:
for ft in shared_fts:
    print(ft)
    sub = train.loc[train["feature_text"]==ft, :]
    sub = sub[["case_num", "pn_history", "feature_text"]].drop_duplicates().reset_index(drop=True)
    display(sub)
    print("List of case_num: ", sub["case_num"].unique())
    # for cn,ph in zip(sub["case_num"], sub["pn_history"]):
    #     print("case_num: ", cn)
    #     print("pn_hitory:\n", ph)
    #     print()
    print()

# Test data
Return to [Header](#Header)

In [ ]:
test = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/test.csv")

In [ ]:
print(test.shape)
test = merge_df(test, ptnote, feat, train=False)
print(test.shape)

In [ ]:
test

In [ ]:
txt = test["pn_history"][0]
for i in range(len(test)):
    assert test["pn_history"][i] == txt

In [ ]:
txt

In [ ]:
ssub = pd.read_csv("/kaggle/input/nbme-score-clinical-patient-notes/sample_submission.csv")
ssub

In [ ]:
txt[200:250]

In [ ]:
txt[300:400]

# patients_note.csv
How many rows in patients_note are not apearing in both train and test?

Return to [Header](#Header)

In [ ]:
ptnote

In [ ]:
train["pn_history"].nunique()

In [ ]:
train["pn_num"].nunique()

In [ ]:
ptnote["isTrain"] = ptnote["pn_num"].apply(lambda x: x in train["pn_num"].unique())
#ptnote["isTest"] = ptnote["pn_num"].apply(lambda x: x in test["pn_num"].unique())

In [ ]:
# ratio of isTrain
ptnote["isTrain"].mean()

In [ ]:
ptnote_isnotTrain = ptnote[ptnote["isTrain"]==0]
ptnote_isTrain = ptnote[ptnote["isTrain"]==1]
ptnote_isnotTrain

In [ ]:
ptnote_isTrain.groupby("case_num")["pn_num"].nunique()

In [ ]:
ptnote_isnotTrain.groupby("case_num")["pn_num"].nunique()

> To help you author submission code, we include a few example instances selected from the training set. When your submitted notebook is scored, this example data will be replaced by the actual test data. The patient notes in the test set will be added to the patient_notes.csv file. These patient notes are from the same clinical cases as the patient notes in the training set. **There are approximately 2000 patient notes in the test set.**

In [ ]:
ptnote_isnotTrain.to_csv("patient_notes_isnotTrain.csv", index=False)